In [1]:
%matplotlib notebook

import os
import collections
import warnings

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import matplotlib as mpl
import matplotlib.dates as mdates
from matplotlib import pylab as plt

import seaborn as sns

import numpy as np
import pandas as pd
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp

from tensorflow_probability import distributions as tfd
from tensorflow_probability import sts

tf.enable_v2_behavior()

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

sns.set_context("notebook", font_scale=1.)
sns.set_style("whitegrid")
%config InlineBackend.figure_format = 'retina'


In [2]:
import cbpro

weburl = 'https://public.sandbox.pro.coinbase.com'
resturl = 'https://api-public.sandbox.pro.coinbase.com'
socketurl = 'wss://ws-feed-public.sandbox.pro.coinbase.com'
fixurl = 'tcp+ssl://fix-public.sandbox.pro.coinbase.com:4198'

passphrase = 'je5zm0rm4i'
key = '5fc3717334a8718cd60f3ee74b819cec'
b64secret = '/fnWX+p82wuYbdjKZ3rlIceuUB1hxut2/6/EVchzuBntYHnZ4PxpcG2epTuvUnV8IAlzGu584mWBPch7wo6Ohw=='

client = cbpro.AuthenticatedClient( key, b64secret, passphrase, api_url=resturl )


In [3]:
client.get_accounts()

[{'id': '44078656-cfca-4355-9ddd-d7f7c927ca9e',
  'currency': 'BTC',
  'balance': '0.2754375000000000',
  'available': '0',
  'hold': '0.2754375000000000',
  'profile_id': 'ba877dcd-bb9f-475d-ba02-fc4a435e2cb4',
  'trading_enabled': True},
 {'id': 'dcf488b7-6e9e-4683-8d71-5e13fc151a2b',
  'currency': 'USDC',
  'balance': '0.0000000000000000',
  'available': '0',
  'hold': '0.0000000000000000',
  'profile_id': 'ba877dcd-bb9f-475d-ba02-fc4a435e2cb4',
  'trading_enabled': True},
 {'id': '3c4528aa-7065-42c8-999b-37f322af98f5',
  'currency': 'USD',
  'balance': '7977.4288199585295000',
  'available': '7977.4288199585295',
  'hold': '0.0000000000000000',
  'profile_id': 'ba877dcd-bb9f-475d-ba02-fc4a435e2cb4',
  'trading_enabled': True},
 {'id': 'e79a4155-bf5a-4b26-8f55-f6e5e7012660',
  'currency': 'LINK',
  'balance': '0.0000000000000000',
  'available': '0',
  'hold': '0.0000000000000000',
  'profile_id': 'ba877dcd-bb9f-475d-ba02-fc4a435e2cb4',
  'trading_enabled': True},
 {'id': 'e8b56bd5-

In [4]:
client.get_account('44078656-cfca-4355-9ddd-d7f7c927ca9e')

{'id': '44078656-cfca-4355-9ddd-d7f7c927ca9e',
 'currency': 'BTC',
 'balance': '0.2754375000000000',
 'available': '0',
 'hold': '0.2754375000000000',
 'profile_id': 'ba877dcd-bb9f-475d-ba02-fc4a435e2cb4',
 'trading_enabled': True}

In [31]:
import time

class myWebsocketClient(cbpro.WebsocketClient):

    def __init__( self ):
        
        self.message_count = 0
        self.url = socketurl
        self.auth = True
        self.api_key = key
        self.api_passphrase = passphrase
        self.api_secret = b64secret
        self.products = [ 'BTC-USD' ]
        
        super().__init__( channels = [ 'ticker', 'status' ] )
        
    def on_open(self):
        
        print("Lets count the messages!")
    
    def on_message(self, msg):
        
        self.message_count += 1
        if 'type' in msg:
            if 'price' in msg:
                print( "Message type:", msg["type"], "\t@ {:.3f}".format(float(msg["price"])) )
            elif msg[ 'type' ] == 'error':
                print( "Error:", msg['message'], "-", msg['reason'] )
        else:
            print( "Message type:", msg )
            
    def on_close(self):
        
        self.message_count = 500
        print("-- Goodbye! --")
        

In [32]:
wsClient = myWebsocketClient()
wsClient.start()
print(wsClient.url, wsClient.products)
while( wsClient.message_count < 10 ):
    time.sleep(1)


Lets count the messages!
wss://ws-feed.pro.coinbase.com ['BTC-USD']
Message type: ticker 	@ 7160.230
Message type: ticker 	@ 7158.010
Message type: ticker 	@ 7158.010
Message type: ticker 	@ 7158.000
Message type: ticker 	@ 7158.010
Message type: ticker 	@ 7158.010
Message type: ticker 	@ 7158.000
Message type: ticker 	@ 7158.010
Message type: ticker 	@ 7158.010
Message type: ticker 	@ 7158.010
Message type: ticker 	@ 7158.010
Message type: ticker 	@ 7158.010
Message type: ticker 	@ 7158.010
Message type: ticker 	@ 7158.000
Message type: ticker 	@ 7156.610
Message type: ticker 	@ 7154.360
Message type: ticker 	@ 7154.360
Message type: ticker 	@ 7154.370
Message type: ticker 	@ 7154.360
Message type: ticker 	@ 7152.560
Message type: ticker 	@ 7152.250
Message type: ticker 	@ 7151.000
Message type: ticker 	@ 7149.980
Message type: ticker 	@ 7149.970
Message type: ticker 	@ 7150.140


In [36]:
wsClient.close()


-- Goodbye! --


In [6]:
f = client.get_fills('44078656-cfca-4355-9ddd-d7f7c927ca9e')
list(f)


['message']

In [7]:
client.get_currencies()


[{'id': 'BAT',
  'name': 'Basic Attention Token',
  'min_size': '1',
  'status': 'online',
  'message': None,
  'details': {'type': 'crypto',
   'symbol': '',
   'network_confirmations': 35,
   'sort_order': 10,
   'crypto_address_link': 'https://etherscan.io/token/0x0d8775f648430679a709e98d2b0cb6250d2887ef?a={{address}}',
   'crypto_transaction_link': 'https://etherscan.io/tx/0x{{txId}}',
   'push_payment_methods': ['crypto']},
  'max_precision': '1'},
 {'id': 'LINK',
  'name': 'Chainlink',
  'min_size': '1',
  'status': 'online',
  'message': None,
  'details': {'type': 'crypto',
   'symbol': 'Ξ',
   'network_confirmations': 35,
   'sort_order': 67,
   'crypto_address_link': 'https://etherscan.io/token/0x514910771af9ca656af840dff83e8264ecf986ca?a={{address}}',
   'crypto_transaction_link': 'https://etherscan.io/tx/0x{{txId}}',
   'push_payment_methods': ['crypto']},
  'max_precision': '0.00000001'},
 {'id': 'USD',
  'name': 'United States Dollar',
  'min_size': '0.01',
  'status': 'o

In [8]:
client.get_products()

[{'id': 'BTC-GBP',
  'base_currency': 'BTC',
  'quote_currency': 'GBP',
  'base_min_size': '0.00100000',
  'base_max_size': '10000.00000000',
  'quote_increment': '0.01000000',
  'base_increment': '0.00000001',
  'display_name': 'BTC/GBP',
  'min_market_funds': '10',
  'max_market_funds': '200000',
  'margin_enabled': False,
  'post_only': False,
  'limit_only': False,
  'cancel_only': False,
  'status': 'online',
  'status_message': ''},
 {'id': 'BTC-EUR',
  'base_currency': 'BTC',
  'quote_currency': 'EUR',
  'base_min_size': '0.00100000',
  'base_max_size': '10000.00000000',
  'quote_increment': '0.01000000',
  'base_increment': '0.00000001',
  'display_name': 'BTC/EUR',
  'min_market_funds': '10',
  'max_market_funds': '600000',
  'margin_enabled': False,
  'post_only': False,
  'limit_only': False,
  'cancel_only': False,
  'status': 'online',
  'status_message': ''},
 {'id': 'BAT-USDC',
  'base_currency': 'BAT',
  'quote_currency': 'USDC',
  'base_min_size': '1.00000000',
  'base

In [9]:
client.get_product_24hr_stats('BTC-USD')

{'open': '8175.05000000',
 'high': '8214.40000000',
 'low': '8032.57000000',
 'volume': '57040.61524600',
 'last': '8082.99000000',
 'volume_30day': '2880081.83929891'}

In [11]:
btcusd=client.get_product_historic_rates('BTC-USD',granularity=60)

In [14]:
len(btcusd)

300

In [16]:
client.get_product_order_book('BTC-USD', level=2)

{'sequence': 88592238,
 'bids': [['7242.99', '1.0914', 1],
  ['7241.5', '0.1', 1],
  ['7241.08', '3.442', 1],
  ['7240.64', '3.45', 1],
  ['7226.31', '0.0301', 1],
  ['7220.86', '0.6123', 1],
  ['7216.65', '10.0502', 1],
  ['7200', '0.0435', 1],
  ['7123.05', '0.00140389', 1],
  ['7110.67', '997.61929805', 1],
  ['7103.99', '0.45914232', 3],
  ['7100', '1', 1],
  ['7051.1', '0.00141821', 1],
  ['7000', '10010', 2],
  ['6996.91', '0.006', 1],
  ['6989.7', '0.1', 1],
  ['6979.15', '0.00143283', 1],
  ['6938.98', '3.347', 2],
  ['6742.49', '10000', 1],
  ['6672.36', '1943.7772', 1],
  ['6661.68', '87', 1],
  ['6661.52', '8.8325', 1],
  ['6660.51', '1124', 1],
  ['6612', '9555', 1],
  ['6474.75', '0.00154446', 1],
  ['6099', '3000', 1],
  ['6065.17', '0.5', 1],
  ['5333.65', '0.0108', 2],
  ['5267', '0.0756', 1],
  ['5200', '4.5364', 1],
  ['5193.32', '0.0112', 2],
  ['5165', '2555', 1],
  ['5100', '0.01', 1],
  ['5052.99', '0.0116', 2],
  ['5000', '1059.1159', 6],
  ['4912.66', '0.012', 2